In [1]:
import tensorflow as tf
import numpy as np

In [3]:
#create real data
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

### create tensorflow structure start ###

#定义变量
Weights = tf.Variable(tf.random_uniform([1],-1.0,1.0))
biases = tf.Variable(tf.zeros([1]))

#如何计算预测值
y = Weights * x_data + biases

# loss function
loss = tf.reduce_mean(tf.square(y-y_data))

#梯度下降优化器，定义learning rate
optimizer = tf.train.GradientDescentOptimizer(0.5)

#训练目标是loss最小化
train = optimizer.minimize(loss)
saver = tf.train.Saver()
#初始化变量，即初始化 Weights 和 biases
init = tf.global_variables_initializer()

#创建session，进行参数初始化
sess = tf.Session()
sess.run(init)

#开始训练200步，每隔20步输出一下两个参数
for step in range(201):
    sess.run(train)
    if step % 20 == 0:
        print(step,sess.run(Weights),sess.run(biases))
### create tensorflow structure end ###

# Save the variables to disk.
save_path = saver.save(sess, "/tmp/tf/model0.ckpt")
print("Model saved in path: %s" % save_path)

0 [0.58753854] [0.03048179]
20 [0.21819495] [0.23390199]
40 [0.12873276] [0.28393182]
60 [0.10698482] [0.2960939]
80 [0.10169799] [0.29905045]
100 [0.10041277] [0.29976916]
120 [0.10010035] [0.2999439]
140 [0.1000244] [0.29998636]
160 [0.10000592] [0.2999967]
180 [0.10000143] [0.2999992]
200 [0.10000035] [0.2999998]
Model saved in path: /tmp/tf/model0.ckpt


In [4]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/tf/model0.ckpt", tensor_name='', all_tensors=True)

tensor_name:  Variable
[0.98456216]
tensor_name:  Variable_1
[0.]
tensor_name:  Variable_2
[0.10000035]
tensor_name:  Variable_3
[0.2999998]
# Total number of params: 4


In [5]:
def predict(sess, inputs):
    preds = sess.run('pred:0', feed_dict={'X:0': inputs}) 
    # `X` is used, it must be defined in the model with that name explicitly!
    return [str(p) for p in preds]

In [6]:
from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers.tensorflow import deploy_tensorflow_model
clipper_conn = ClipperConnection(DockerContainerManager())

In [7]:
clipper_conn.start_clipper()

20-01-10:02:19:55 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-01-10:02:19:59 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmpqxmbo9k2.yml
20-01-10:02:20:00 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [8]:
clipper_conn.connect()

20-01-10:02:20:22 INFO     [clipper_admin.py:172] [default-cluster] Successfully connected to Clipper cluster at localhost:1337


In [9]:
# Add an application with a name and an input type
clipper_conn.register_application(
    name="tf-app0", input_type="doubles", default_output="-1.0", slo_micros=100000)

20-01-10:02:23:18 INFO     [clipper_admin.py:236] [default-cluster] Application tf-app0 was successfully registered


In [10]:

# Deploy a model, to check what arguments you need run 
# "?deploy_tensorflow_model" to let the notebook show you the definition of the method
deploy_tensorflow_model(
    clipper_conn,
    name="tf-mod0",
    version=1,
    input_type="doubles",
    func=predict,
    tf_sess_or_saved_model_path="/tmp/tf/") # path to saved model files is used,
# session example in the next section

20-01-10:02:26:41 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpwgbtnss1clipper
20-01-10:02:26:41 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-01-10:02:26:41 INFO     [tensorflow.py:264] TensorFlow model copied to: tfmodel 
20-01-10:02:26:41 INFO     [tensorflow.py:277] Using Python 3.6 base image
20-01-10:02:26:41 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpwgbtnss1clipper
20-01-10:02:26:41 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/tf36-container:0.4.1
20-01-10:02:26:41 INFO     [clipper_admin.py:539] [default-cluster]  ---> 3db42af800ff
20-01-10:02:26:41 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmpwgbtnss1clipper /model/
20-01-10:02:26:41 INFO     [clipper_admin.py:539] [default-cluster]  ---> 3644abd6bb34
20-01-10:02:26:41 INFO     [clipper_admin.py:539] [default-cluster] Successfully built 3644abd6bb34
20-01-10:02:2

In [11]:
# Link the model and the app
clipper_conn.link_model_to_app(
    app_name="tf-app0",
    model_name="tf-mod0")

20-01-10:02:30:21 INFO     [clipper_admin.py:303] [default-cluster] Model tf-mod0 is now linked to application tf-app0


In [12]:
# Get query address
query_address = clipper_conn.get_query_addr()

In [16]:
# Run a query
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://"+query_address+"/tf-app0/predict", headers=headers, data=json.dumps({
    "input": [.8,3.2]})).json()

{'query_id': 2, 'output': '[0.6076976 1.3544037]', 'default': False}

In [17]:
clipper_conn.unlink_model_from_app(model_name="tf-mod0", app_name="tf-app0")

20-01-10:06:51:35 INFO     [clipper_admin.py:344] Model tf-mod0 is now removed to application tf-app0


In [18]:
clipper_conn.stop_models('tf-mod0')

20-01-10:06:52:05 INFO     [clipper_admin.py:1285] Model tf-mod0:1 was successfully deleted
20-01-10:06:52:05 INFO     [clipper_admin.py:1319] [default-cluster] Stopped all containers for these models and versions:
{'tf-mod0': ['1']}


In [19]:
clipper_conn.delete_application('tf-app0')

20-01-10:06:52:18 WARNING  [clipper_admin.py:240] [default-cluster] [DEPRECATED] Use 'unregister_application' API instead of this.
20-01-10:06:52:18 INFO     [clipper_admin.py:260] [default-cluster] Application tf-app0 was successfully unregistered


In [20]:
clipper_conn.stop_all()

20-01-10:06:53:04 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers
